In [1]:
from sympy import *
from sympy.physics.quantum import *
import pandas as pd
from scipy.sparse import csc_matrix

# Monolayer

In [2]:
ket={(kind,l,m):Ket(r'{},{},{}'.format(kind,l,m)) for kind,l in zip('MAB',(2,1,1)) for m in range(-l,l+1)}

In [3]:
bra={(kind,l,m):Bra(r'{},{},{}'.format(kind,l,m)) for kind,l in zip('MAB',(2,1,1)) for m in range(-l,l+1)}

In [4]:
L={(kind,ladder):Operator('L^{}_{}'.format(kind,ladder)) for kind in 'MAB' for ladder in '-+z'}

In [5]:
k={}
k[1]=-1/sqrt(2)*(ket[('M',2,1)]-ket[('M',2,-1)])
k[2]=I/sqrt(2)*(ket[('M',2,1)]+ket[('M',2,-1)])
k[3]=1/sqrt(2)*(ket[('A',1,0)]+ket[('B',1,0)])
k[4]=1/sqrt(2)*((-1/sqrt(2))*(ket[('A',1,1)]-ket[('A',1,-1)])-(-1/sqrt(2))*(ket[('B',1,1)]-ket[('B',1,-1)]))
k[5]=1/sqrt(2)*(I/sqrt(2)*(ket[('A',1,1)]+ket[('A',1,-1)])-I/sqrt(2)*(ket[('B',1,1)]+ket[('B',1,-1)]))
k[6]=ket[('M',2,0)]
k[7]=-I/sqrt(2)*(ket[('M',2,2)]-ket[('M',2,-2)])
k[8]=1/sqrt(2)*(ket[('M',2,2)]+ket[('M',2,-2)])
k[9]=1/sqrt(2)*(ket[('A',1,0)]-ket[('B',1,0)])
k[10]=1/sqrt(2)*((-1/sqrt(2))*(ket[('A',1,1)]-ket[('A',1,-1)])+(-1/sqrt(2))*(ket[('B',1,1)]-ket[('B',1,-1)]))
k[11]=1/sqrt(2)*(I/sqrt(2)*(ket[('A',1,1)]+ket[('A',1,-1)])+I/sqrt(2)*(ket[('B',1,1)]+ket[('B',1,-1)]))

In [6]:
def func(op,state):
    kind_op,action=op
    kind_state,l,m=state
    assert kind_op in 'MAB', 'Op ({}) not recognized'.format(kind_op)
    assert kind_state in 'MAB', 'State ({}) not recognized'.format(kind_state)
    assert action in '-+z', 'Op action ({}) not recognized'.format(action)
    if kind_op==kind_state:
        if action=='z':
            return m*ket[state]
        if action=='-':
            if m==-l:
                return 0
            else:
                return sqrt(l*(l+1)-m*(m-1))*ket[(kind_state,l,m-1)]
        if action=='+':
            if m==l:
                return 0
            else:
                return sqrt(l*(l+1)-m*(m+1))*ket[(kind_state,l,m+1)]
    else:
        return 0
    

In [7]:
rule_L=[(op_val*state_val,func(op_key,state_key)) for op_key,op_val in L.items() for state_key,state_val in ket.items()]

In [8]:
innerprod=[(bra[label1]*ket[label2],1 if label1 == label2 else 0) for label1 in ket.keys() for label2 in ket.keys()]

In [9]:
Lmat={}
Lmat={key:Matrix([[qapply((Dagger(s_b)*val*s_k).expand().subs(rule_L)).subs(innerprod) for s_k in list(k.values())] for s_b in list(k.values())]) for key,val in L.items()}

In [59]:
S={'+':Matrix([[0,1],[0,0]])/2,'-':Matrix([[0,0],[1,0]])/2,'z':Matrix([[1,0],[0,-1]])/2}

In [60]:
SOC={atom:(TensorProduct(Lmat[(atom,'-')],S['+'])+TensorProduct(Lmat[(atom,'+')],S['-']))/2+TensorProduct(Lmat[(atom,'z')],S['z']) for atom in 'MAB'}

In [61]:
basis=['{}{}'.format(i,spin) for i in range(1,12) for spin in '↑↓' ]

In [62]:
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 100)

In [63]:
pd.DataFrame(SOC['M'].tolist(),index=basis, columns=basis)

,1↑,1↓,2↑,2↓,3↑,3↓,4↑,4↓,5↑,5↓,6↑,6↓,7↑,7↓,8↑,8↓,9↑,9↓,10↑,10↓,11↑,11↓
1↑,0,0,-I/2,0,0,0,0,0,0,0,0,sqrt(3)/4,0,I/4,0,-1/4,0,0,0,0,0,0
1↓,0,0,0,I/2,0,0,0,0,0,0,-sqrt(3)/4,0,I/4,0,1/4,0,0,0,0,0,0,0
2↑,I/2,0,0,0,0,0,0,0,0,0,0,-sqrt(3)*I/4,0,-1/4,0,-I/4,0,0,0,0,0,0
2↓,0,-I/2,0,0,0,0,0,0,0,0,-sqrt(3)*I/4,0,1/4,0,-I/4,0,0,0,0,0,0,0
3↑,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3↓,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4↑,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4↓,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5↑,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5↓,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
SOC['A']

Matrix([
[0, 0, 0, 0,    0,    0,   0,    0,    0,   0, 0, 0, 0, 0, 0, 0,    0,    0,   0,    0,    0,   0],
[0, 0, 0, 0,    0,    0,   0,    0,    0,   0, 0, 0, 0, 0, 0, 0,    0,    0,   0,    0,    0,   0],
[0, 0, 0, 0,    0,    0,   0,    0,    0,   0, 0, 0, 0, 0, 0, 0,    0,    0,   0,    0,    0,   0],
[0, 0, 0, 0,    0,    0,   0,    0,    0,   0, 0, 0, 0, 0, 0, 0,    0,    0,   0,    0,    0,   0],
[0, 0, 0, 0,    0,    0,   0, -1/8,    0, I/8, 0, 0, 0, 0, 0, 0,    0,    0,   0, -1/8,    0, I/8],
[0, 0, 0, 0,    0,    0, 1/8,    0,  I/8,   0, 0, 0, 0, 0, 0, 0,    0,    0, 1/8,    0,  I/8,   0],
[0, 0, 0, 0,    0,  1/8,   0,    0, -I/4,   0, 0, 0, 0, 0, 0, 0,    0,  1/8,   0,    0, -I/4,   0],
[0, 0, 0, 0, -1/8,    0,   0,    0,    0, I/4, 0, 0, 0, 0, 0, 0, -1/8,    0,   0,    0,    0, I/4],
[0, 0, 0, 0,    0, -I/8, I/4,    0,    0,   0, 0, 0, 0, 0, 0, 0,    0, -I/8, I/4,    0,    0,   0],
[0, 0, 0, 0, -I/8,    0,   0, -I/4,    0,   0, 0, 0, 0, 0, 0, 0, -I/8,    0,   0, -I/4,    

In [16]:
pd.DataFrame(SOC['A'].tolist(),index=basis, columns=basis)

,1↑,1↓,2↑,2↓,3↑,3↓,4↑,4↓,5↑,5↓,6↑,6↓,7↑,7↓,8↑,8↓,9↑,9↓,10↑,10↓,11↑,11↓
1↑,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1↓,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2↑,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2↓,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3↑,0,0,0,0,0,0,0,-1/8,0,I/8,0,0,0,0,0,0,0,0,0,-1/8,0,I/8
3↓,0,0,0,0,0,0,1/8,0,I/8,0,0,0,0,0,0,0,0,0,1/8,0,I/8,0
4↑,0,0,0,0,0,1/8,0,0,-I/4,0,0,0,0,0,0,0,0,1/8,0,0,-I/4,0
4↓,0,0,0,0,-1/8,0,0,0,0,I/4,0,0,0,0,0,0,-1/8,0,0,0,0,I/4
5↑,0,0,0,0,0,-I/8,I/4,0,0,0,0,0,0,0,0,0,0,-I/8,I/4,0,0,0
5↓,0,0,0,0,-I/8,0,0,-I/4,0,0,0,0,0,0,0,0,-I/8,0,0,-I/4,0,0


In [17]:
pd.DataFrame(SOC['B'].tolist(),index=basis, columns=basis)

,1↑,1↓,2↑,2↓,3↑,3↓,4↑,4↓,5↑,5↓,6↑,6↓,7↑,7↓,8↑,8↓,9↑,9↓,10↑,10↓,11↑,11↓
1↑,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1↓,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2↑,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2↓,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3↑,0,0,0,0,0,0,0,1/8,0,-I/8,0,0,0,0,0,0,0,0,0,-1/8,0,I/8
3↓,0,0,0,0,0,0,-1/8,0,-I/8,0,0,0,0,0,0,0,0,0,1/8,0,I/8,0
4↑,0,0,0,0,0,-1/8,0,0,-I/4,0,0,0,0,0,0,0,0,1/8,0,0,I/4,0
4↓,0,0,0,0,1/8,0,0,0,0,I/4,0,0,0,0,0,0,-1/8,0,0,0,0,-I/4
5↑,0,0,0,0,0,I/8,I/4,0,0,0,0,0,0,0,0,0,0,-I/8,-I/4,0,0,0
5↓,0,0,0,0,I/8,0,0,-I/4,0,0,0,0,0,0,0,0,-I/8,0,0,I/4,0,0


In [18]:
SOC['A']+SOC['B']

Matrix([
[0, 0, 0, 0,    0,    0,   0,    0,    0,   0, 0, 0, 0, 0, 0, 0,    0,    0,   0,    0,    0,   0],
[0, 0, 0, 0,    0,    0,   0,    0,    0,   0, 0, 0, 0, 0, 0, 0,    0,    0,   0,    0,    0,   0],
[0, 0, 0, 0,    0,    0,   0,    0,    0,   0, 0, 0, 0, 0, 0, 0,    0,    0,   0,    0,    0,   0],
[0, 0, 0, 0,    0,    0,   0,    0,    0,   0, 0, 0, 0, 0, 0, 0,    0,    0,   0,    0,    0,   0],
[0, 0, 0, 0,    0,    0,   0,    0,    0,   0, 0, 0, 0, 0, 0, 0,    0,    0,   0, -1/4,    0, I/4],
[0, 0, 0, 0,    0,    0,   0,    0,    0,   0, 0, 0, 0, 0, 0, 0,    0,    0, 1/4,    0,  I/4,   0],
[0, 0, 0, 0,    0,    0,   0,    0, -I/2,   0, 0, 0, 0, 0, 0, 0,    0,  1/4,   0,    0,    0,   0],
[0, 0, 0, 0,    0,    0,   0,    0,    0, I/2, 0, 0, 0, 0, 0, 0, -1/4,    0,   0,    0,    0,   0],
[0, 0, 0, 0,    0,    0, I/2,    0,    0,   0, 0, 0, 0, 0, 0, 0,    0, -I/4,   0,    0,    0,   0],
[0, 0, 0, 0,    0,    0,   0, -I/2,    0,   0, 0, 0, 0, 0, 0, 0, -I/4,    0,   0,    0,    

# Print matrix index

In [111]:
def print_(mat):
    row_list=[]
    col_list=[]
    val_list=[]
    for i in range(22):
        for j in range(22):
            val=mat[i,j]
            if val!=0:
                row_list.append(i)
                col_list.append(j)
                val_list.append(val)
    return row_list,col_list,val_list


In [112]:
row_list,col_list,val_list=print_(SOC['M'])

In [113]:
print(row_list)
print(col_list)
print(val_list)

[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 10, 10, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15]
[2, 11, 13, 15, 3, 10, 12, 14, 0, 11, 13, 15, 1, 10, 12, 14, 1, 3, 0, 2, 1, 3, 14, 0, 2, 15, 1, 3, 12, 0, 2, 13]
[-I/2, sqrt(3)/4, I/4, -1/4, I/2, -sqrt(3)/4, I/4, 1/4, I/2, -sqrt(3)*I/4, -1/4, -I/4, -I/2, -sqrt(3)*I/4, 1/4, -I/4, -sqrt(3)/4, sqrt(3)*I/4, sqrt(3)/4, sqrt(3)*I/4, -I/4, 1/4, I, -I/4, -1/4, -I, 1/4, I/4, -I, -1/4, I/4, I]


In [114]:
r_M=[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 10, 10, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15]
c_M=[2, 11, 13, 15, 3, 10, 12, 14, 0, 11, 13, 15, 1, 10, 12, 14, 1, 3, 0, 2, 1, 3, 14, 0, 2, 15, 1, 3, 12, 0, 2, 13]
v_M=[-1j/2, np.sqrt(3)/4, 1j/4, -1/4, 1j/2, -np.sqrt(3)/4, 1j/4, 1/4, 1j/2, -np.sqrt(3)*1j/4, -1/4, -1j/4, -1j/2, -np.sqrt(3)*1j/4, 1/4, -1j/4, -np.sqrt(3)/4, np.sqrt(3)*1j/4, np.sqrt(3)/4, np.sqrt(3)*1j/4, -1j/4, 1/4, 1j, -1j/4, -1/4, -1j, 1/4, 1j/4, -1j, -1/4, 1j/4, 1j]

In [115]:
SOC_M=csc_matrix((v_M,(r_M,c_M)),(22,22))

In [116]:
Matrix(SOC_M.toarray().tolist())-N(SOC['M'])

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [117]:
row_list,col_list,val_list=print_(SOC['A'])

In [118]:
print(row_list)
print(col_list)
print(val_list)

[4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 16, 16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 19, 19, 19, 19, 20, 20, 20, 20, 21, 21, 21, 21]
[7, 9, 19, 21, 6, 8, 18, 20, 5, 8, 17, 20, 4, 9, 16, 21, 5, 6, 17, 18, 4, 7, 16, 19, 7, 9, 19, 21, 6, 8, 18, 20, 5, 8, 17, 20, 4, 9, 16, 21, 5, 6, 17, 18, 4, 7, 16, 19]
[-1/8, I/8, -1/8, I/8, 1/8, I/8, 1/8, I/8, 1/8, -I/4, 1/8, -I/4, -1/8, I/4, -1/8, I/4, -I/8, I/4, -I/8, I/4, -I/8, -I/4, -I/8, -I/4, -1/8, I/8, -1/8, I/8, 1/8, I/8, 1/8, I/8, 1/8, -I/4, 1/8, -I/4, -1/8, I/4, -1/8, I/4, -I/8, I/4, -I/8, I/4, -I/8, -I/4, -I/8, -I/4]


In [119]:
r_A=[4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 16, 16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 19, 19, 19, 19, 20, 20, 20, 20, 21, 21, 21, 21]
c_A=[7, 9, 19, 21, 6, 8, 18, 20, 5, 8, 17, 20, 4, 9, 16, 21, 5, 6, 17, 18, 4, 7, 16, 19, 7, 9, 19, 21, 6, 8, 18, 20, 5, 8, 17, 20, 4, 9, 16, 21, 5, 6, 17, 18, 4, 7, 16, 19]
v_A=[-1/8, 1j/8, -1/8, 1j/8, 1/8, 1j/8, 1/8, 1j/8, 1/8, -1j/4, 1/8, -1j/4, -1/8, 1j/4, -1/8, 1j/4, -1j/8, 1j/4, -1j/8, 1j/4, -1j/8, -1j/4, -1j/8, -1j/4, -1/8, 1j/8, -1/8, 1j/8, 1/8, 1j/8, 1/8, 1j/8, 1/8, -1j/4, 1/8, -1j/4, -1/8, 1j/4, -1/8, 1j/4, -1j/8, 1j/4, -1j/8, 1j/4, -1j/8, -1j/4, -1j/8, -1j/4]

In [120]:
SOC_A=csc_matrix((v_A,(r_A,c_A)),(22,22))

In [121]:
Matrix(SOC_A.toarray().tolist())-N(SOC['A'])

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [126]:
row_list,col_list,val_list=print_(SOC['B'])

In [127]:
print(row_list)
print(col_list)
print(val_list)

[4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 16, 16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 19, 19, 19, 19, 20, 20, 20, 20, 21, 21, 21, 21]
[7, 9, 19, 21, 6, 8, 18, 20, 5, 8, 17, 20, 4, 9, 16, 21, 5, 6, 17, 18, 4, 7, 16, 19, 7, 9, 19, 21, 6, 8, 18, 20, 5, 8, 17, 20, 4, 9, 16, 21, 5, 6, 17, 18, 4, 7, 16, 19]
[1/8, -I/8, -1/8, I/8, -1/8, -I/8, 1/8, I/8, -1/8, -I/4, 1/8, I/4, 1/8, I/4, -1/8, -I/4, I/8, I/4, -I/8, -I/4, I/8, -I/4, -I/8, I/4, -1/8, I/8, 1/8, -I/8, 1/8, I/8, -1/8, -I/8, 1/8, I/4, -1/8, -I/4, -1/8, -I/4, 1/8, I/4, -I/8, -I/4, I/8, I/4, -I/8, I/4, I/8, -I/4]


In [128]:
r_B=[4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 16, 16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 19, 19, 19, 19, 20, 20, 20, 20, 21, 21, 21, 21]
c_B=[7, 9, 19, 21, 6, 8, 18, 20, 5, 8, 17, 20, 4, 9, 16, 21, 5, 6, 17, 18, 4, 7, 16, 19, 7, 9, 19, 21, 6, 8, 18, 20, 5, 8, 17, 20, 4, 9, 16, 21, 5, 6, 17, 18, 4, 7, 16, 19]
v_B=[1/8, -1j/8, -1/8, 1j/8, -1/8, -1j/8, 1/8, 1j/8, -1/8, -1j/4, 1/8, 1j/4, 1/8, 1j/4, -1/8, -1j/4, 1j/8, 1j/4, -1j/8, -1j/4, 1j/8, -1j/4, -1j/8, 1j/4, -1/8, 1j/8, 1/8, -1j/8, 1/8, 1j/8, -1/8, -1j/8, 1/8, 1j/4, -1/8, -1j/4, -1/8, -1j/4, 1/8, 1j/4, -1j/8, -1j/4, 1j/8, 1j/4, -1j/8, 1j/4, 1j/8, -1j/4]

In [129]:
SOC_B=csc_matrix((v_B,(r_B,c_B)),(22,22))

In [130]:
Matrix(SOC_B.toarray().tolist())-N(SOC['B'])

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
Sum((TensorProduct(Lmat[(kind,'-')],S['+'])+TensorProduct(Lmat[(kind,'+')],S['-']))/2+TensorProduct(Lmat[(kind,'z')],S['z']),(kind,'MAB'))

# Bilayer

In [23]:
ket_2={(kind,l,m,layer):Ket(r'{},{},{}'.format(kind,l,m)) for layer in 'bt' for kind,l in zip('MAB',(2,1,1)) for m in range(-l,l+1)}

In [24]:
bra_2={(kind,l,m,layer):Bra(r'{},{},{}'.format(kind,l,m)) for layer in 'bt' for kind,l in zip('MAB',(2,1,1)) for m in range(-l,l+1)}

In [25]:
L_2={(kind,ladder,layer):Operator(r'L^{}_{{{},{}}}'.format(kind,ladder,layer)) for layer in 'bt' for kind in 'MAB' for ladder in '-+z'}

In [28]:
k_2={}
for idx,layer in enumerate('bt'):
    k_2[1+11*idx]=-1/sqrt(2)*(ket_2[('M',2,1,layer)]-ket_2[('M',2,-1,layer)])
    k_2[2+11*idx]=I/sqrt(2)*(ket_2[('M',2,1,layer)]+ket_2[('M',2,-1,layer)])
    k_2[3+11*idx]=1/sqrt(2)*(ket_2[('A',1,0,layer)]+ket_2[('B',1,0,layer)])
    k_2[4+11*idx]=1/sqrt(2)*((-1/sqrt(2))*(ket_2[('A',1,1,layer)]-ket_2[('A',1,-1,layer)])-(-1/sqrt(2))*(ket_2[('B',1,1,layer)]-ket_2[('B',1,-1,layer)]))
    k_2[5+11*idx]=1/sqrt(2)*(I/sqrt(2)*(ket_2[('A',1,1,layer)]+ket_2[('A',1,-1,layer)])-I/sqrt(2)*(ket_2[('B',1,1,layer)]+ket_2[('B',1,-1,layer)]))
    k_2[6+11*idx]=ket_2[('M',2,0,layer)]
    k_2[7+11*idx]=-I/sqrt(2)*(ket_2[('M',2,2,layer)]-ket_2[('M',2,-2,layer)])
    k_2[8+11*idx]=1/sqrt(2)*(ket_2[('M',2,2,layer)]+ket_2[('M',2,-2,layer)])
    k_2[9+11*idx]=1/sqrt(2)*(ket_2[('A',1,0,layer)]-ket_2[('B',1,0,layer)])
    k_2[10+11*idx]=1/sqrt(2)*((-1/sqrt(2))*(ket_2[('A',1,1,layer)]-ket_2[('A',1,-1,layer)])+(-1/sqrt(2))*(ket_2[('B',1,1,layer)]-ket_2[('B',1,-1,layer)]))
    k_2[11+11*idx]=1/sqrt(2)*(I/sqrt(2)*(ket_2[('A',1,1,layer)]+ket_2[('A',1,-1,layer)])+I/sqrt(2)*(ket_2[('B',1,1,layer)]+ket_2[('B',1,-1,layer)]))

In [35]:
def func_2(op,state):
    kind_op,action,layer_op=op
    kind_state,l,m,layer_state=state
    assert kind_op in 'MAB', 'Op ({}) not recognized'.format(kind_op)
    assert kind_state in 'MAB', 'State ({}) not recognized'.format(kind_state)
    assert layer_op in 'tb', 'layer_Op ({}) not recognized'.format(layer_op)
    assert layer_state in 'tb', 'layer_state ({}) not recognized'.format(layer_state)
    assert action in '-+z', 'Op action ({}) not recognized'.format(action)
    if kind_op==kind_state and layer_op==layer_state:
        if action=='z':
            return m*ket_2[state]
        if action=='-':
            if m==-l:
                return 0
            else:
                return sqrt(l*(l+1)-m*(m-1))*ket_2[(kind_state,l,m-1,layer_state)]
        if action=='+':
            if m==l:
                return 0
            else:
                return sqrt(l*(l+1)-m*(m+1))*ket_2[(kind_state,l,m+1,layer_state)]
    else:
        return 0
    

In [36]:
rule_L_2=[(op_val*state_val,func_2(op_key,state_key)) for op_key,op_val in L_2.items() for state_key,state_val in ket_2.items()]

In [37]:
innerprod_2=[(bra_2[label1]*ket_2[label2],1 if label1 == label2 else 0) for label1 in bra_2.keys() for label2 in ket_2.keys()]

In [40]:
Lmat_2={}
Lmat_2={key:Matrix([[qapply((Dagger(s_b)*val*s_k).expand().subs(rule_L_2)).subs(innerprod_2) for s_k in list(k_2.values())] for s_b in list(k_2.values())]) for key,val in L_2.items()}

In [44]:
SOC_2={atom:(TensorProduct(Lmat_2[(atom,'-',layer)],S['+'])+TensorProduct(Lmat_2[(atom,'+',layer)],S['-']))/2+TensorProduct(Lmat_2[(atom,'z',layer)],S['z']) for layer in 'bt' for atom in 'MAB'}

In [49]:
basis_2=['{}{}{}'.format(i,spin,layer) for layer in 'bt' for i in range(1,12) for spin in '↑↓' ]

In [58]:
SOC_2['B'][:22,:22]

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [50]:
pd.DataFrame(SOC_2['M'].tolist(),index=basis_2, columns=basis_2)

,1↑b,1↓b,2↑b,2↓b,3↑b,3↓b,4↑b,4↓b,5↑b,5↓b,6↑b,6↓b,7↑b,7↓b,8↑b,8↓b,9↑b,9↓b,10↑b,10↓b,11↑b,11↓b,1↑t,1↓t,2↑t,2↓t,3↑t,3↓t,4↑t,4↓t,5↑t,5↓t,6↑t,6↓t,7↑t,7↓t,8↑t,8↓t,9↑t,9↓t,10↑t,10↓t,11↑t,11↓t
1↑b,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1↓b,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2↑b,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2↓b,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3↑b,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3↓b,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4↑b,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4↓b,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5↑b,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5↓b,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Test

In [249]:
Lmat[('B','-')].H-Lmat[('B','+')]

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [258]:
pd.DataFrame(Lmat[('M','+')].T.tolist(),columns=range(1,12),index=range(1,12))

,1,2,3,4,5,6,7,8,9,10,11
1,0,0,0,0,0,-sqrt(3),I,1,0,0,0
2,0,0,0,0,0,-sqrt(3)*I,1,-I,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0
6,sqrt(3),sqrt(3)*I,0,0,0,0,0,0,0,0,0
7,-I,-1,0,0,0,0,0,0,0,0,0
8,-1,I,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,0


In [264]:
Lmat[('M','+')].T[[0,1,5,6,7],[0,1,5,6,7]]

Matrix([
[      0,         0,   -sqrt(3), I,  1],
[      0,         0, -sqrt(3)*I, 1, -I],
[sqrt(3), sqrt(3)*I,          0, 0,  0],
[     -I,        -1,          0, 0,  0],
[     -1,         I,          0, 0,  0]])

In [265]:
Lmat[('M','-')].T[[0,1,5,6,7],[0,1,5,6,7]]

Matrix([
[       0,         0,    sqrt(3),  I, -1],
[       0,         0, -sqrt(3)*I, -1, -I],
[-sqrt(3), sqrt(3)*I,          0,  0,  0],
[      -I,         1,          0,  0,  0],
[       1,         I,          0,  0,  0]])

In [266]:
Lmat[('M','z')].T[[0,1,5,6,7],[0,1,5,6,7]]

Matrix([
[0, -I, 0,    0,   0],
[I,  0, 0,    0,   0],
[0,  0, 0,    0,   0],
[0,  0, 0,    0, 2*I],
[0,  0, 0, -2*I,   0]])

In [267]:
pd.DataFrame(Lmat[('A','z')].T.tolist(),columns=range(1,12),index=range(1,12))

,1,2,3,4,5,6,7,8,9,10,11
1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,-I/2,0,0,0,0,0,-I/2
5,0,0,0,I/2,0,0,0,0,0,I/2,0
6,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,-I/2,0,0,0,0,0,-I/2


In [305]:
Lmat[('A','-')].T[[2,3,4,8,9,10],[2,3,4,8,9,10]]

Matrix([
[   0, -1/2, I/2,    0, -1/2, I/2],
[ 1/2,    0,   0,  1/2,    0,   0],
[-I/2,    0,   0, -I/2,    0,   0],
[   0, -1/2, I/2,    0, -1/2, I/2],
[ 1/2,    0,   0,  1/2,    0,   0],
[-I/2,    0,   0, -I/2,    0,   0]])

In [307]:
Lmat[('B','-')].T[[2,3,4,8,9,10],[2,3,4,8,9,10]]

Matrix([
[   0,  1/2, -I/2,    0, -1/2,  I/2],
[-1/2,    0,    0,  1/2,    0,    0],
[ I/2,    0,    0, -I/2,    0,    0],
[   0, -1/2,  I/2,    0,  1/2, -I/2],
[ 1/2,    0,    0, -1/2,    0,    0],
[-I/2,    0,    0,  I/2,    0,    0]])

In [218]:
Matrix(LMm_mat).adjoint()-Matrix(LMp_mat)

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [131]:
rule_L=[(L[('M','-')]*val,func('M-',key)) for key,val in ket.items()]

{('M', '-'): L^M_-,
 ('M', '+'): L^M_+,
 ('M', 'z'): L^M_z,
 ('A', '-'): L^A_-,
 ('A', '+'): L^A_+,
 ('A', 'z'): L^A_z,
 ('B', '-'): L^B_-,
 ('B', '+'): L^B_+,
 ('B', 'z'): L^B_z}

In [130]:
qapply(Dagger(k[1])*k[1]).subs(innerprod)

1

In [104]:
basis=[(k[1],-1/sqrt(2)*(ket[('M',2,1)]-ket[('M',2,-1)])),
(k[2],I/sqrt(2)*(ket[('M',2,1)]+ket[('M',2,-1)])),
]
[]

In [124]:
((-1/sqrt(2)*(bra[('M',2,1)]-bra[('M',2,-1)]))*(-1/sqrt(2)*(ket[('M',2,1)]-ket[('M',2,-1)]))).expand()

<M,2,-1|*|M,2,-1>/2 - <M,2,-1|*|M,2,1>/2 - <M,2,1|*|M,2,-1>/2 + <M,2,1|*|M,2,1>/2

In [126]:
qapply(Dagger(k[1])*k[1]).subs(innerprod)

1

In [37]:
(ket[('M',2,-2)].dual*ket[('M',2,-2)]).doit()

1

In [29]:
L[('M','-')]*ket[('M',2,-2)]

L^M_-*|M,2,-2>

In [84]:
(bra[('M',2,2)]*ket[('M',2,-2)]).doit()

<M,2,2|M,2,-2>

In [89]:
(1,2) == (2,3)

False

In [ ]:
innerprod

In [80]:
(OrthogonalBra((2,1))*OrthogonalKet((2,0))).doit()

TypeError: unsupported operand type(s) for -: 'Tuple' and 'Tuple'

In [95]:
qapply((bra[('M',2,-2)]*(ket[('M',2,-2)]+ket[('M',2,-1)]))).subs(innerprod)

1

In [9]:
ket['M',2,-2]

$\rangle{M,2,-2}$

In [ ]:
L={(l,m):Operator(r'$\rangle{d_{l,m}}$'.format(i)) for i in range(2)}